# SVM and analisys texts

Вы научитесь:

    находить оптимальные параметры для метода опорных векторов
    работать с текстовыми данными

Введение

Метод опорных векторов (Support Vector Machine, SVM) — один из видов линейных классификаторов. Функционал, который он оптимизирует, направлен на максимизацию ширины разделяющей полосы между классами. Из теории статистического обучения известно, что эта ширина тесно связана с обобщающей способностью алгоритма, а ее максимизация позволяет бороться с переобучением.

Одна из причин популярности линейных методов заключается в том, что они хорошо работают на разреженных данных. Так называются выборки с большим количеством признаков, где на каждом объекте большинство признаков равны нулю. Разреженные данные возникают, например, при работе с текстами. Дело в том, что текст удобно кодировать с помощью "мешка слов" — формируется столько признаков, сколько всего уникальных слов встречается в текстах, и значение каждого признака равно числу вхождений в документ соответствующего слова. Ясно, что общее число различных слов в наборе текстов может достигать десятков тысяч, и при этом лишь небольшая их часть будет встречаться в одном конкретном тексте.

Можно кодировать тексты хитрее, и записывать не количество вхождений слова в текст, а TF-IDF. Это показатель, который равен произведению двух чисел: TF (term frequency) и IDF (inverse document frequency). Первая равна отношению числа вхождений слова в документ к общей длине документа. Вторая величина зависит от того, в скольки документах выборки встречается это слово. Чем больше таких документов, тем меньше IDF. Таким образом, TF-IDF будет иметь высокое значение для тех слов, которые много раз встречаются в данном документе, и редко встречаются в остальных.
Данные

Как мы уже говорили выше, линейные методы часто применяются для решения различных задач анализа текстов. В этом задании мы применим метод опорных векторов для определения того, к какой из тематик относится новость: атеизм или космос.

Для начала вам потребуется загрузить данные. В этом задании мы воспользуемся одним из датасетов, доступных в scikit-learn'е — 20 newsgroups. Для этого нужно воспользоваться модулем datasets:
    from sklearn import datasets

    newsgroups = datasets.fetch_20newsgroups(
                        subset='all', 
                        categories=['alt.atheism', 'sci.space']
                 )
Одна из сложностей работы с текстовыми данными состоит в том, что для них нужно построить числовое представление. Одним из способов нахождения такого представления является вычисление TF-IDF. В Scikit-Learn это реализовано в классе sklearn.feature_extraction.text.TfidfVectorizer. Преобразование обучающей выборки нужно делать с помощью функции fit_transform, тестовой — с помощью transform.

Реализация SVM-классификатора находится в классе sklearn.svm.SVC. Веса каждого признака у обученного классификатора хранятся в поле coef_. Чтобы понять, какому слову соответствует i-й признак, можно воспользоваться методом get_feature_names() у TfidfVectorizer:

    feature_mapping = vectorizer.get_feature_names()
    print feature_mapping[i]
Подбор параметров удобно делать с помощью класса sklearn.grid_search.GridSearchCV (При использовании библиотеки scikit-learn версии 18.0.1 sklearn.model_selection.GridSearchCV). Пример использования:

    grid = {'C': np.power(10.0, np.arange(-5, 6))}
    cv = KFold(y.size, n_folds=5, shuffle=True, random_state=241)
    clf = svm.SVC(kernel='linear', random_state=241)
    gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
    gs.fit(X, y)
При использовании библиотеки scikit-learn версии 18.0.1 и выше KFold задаётся немного по-другому:

    cv = KFold(n_splits=5, shuffle=True, random_state=241)
Первым аргументом в GridSearchCV передается классификатор, для которого будут подбираться значения параметров, вторым — словарь (dict), задающий сетку параметров для перебора. После того, как перебор окончен, можно проанализировать значения качества для всех значений параметров и выбрать наилучший вариант:
 
     for a in gs.grid_scores_:
            a.mean_validation_score — оценка качества по кросс-валидации
            a.parameters — значения параметров

Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

In [3]:
from sklearn import datasets
newgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])

In [14]:
import numpy as np
X = np.array(newgroups.data)
y = np.array(newgroups.target)
print(y[:4])
print(X[2][:100])
print(newgroups.target_names)
print(X.shape)

[0 0 1 1]
From: fairfiel@helios.usq.EDU.AU (raymond fairfield)
Subject: Re: Surviving Large Accelerations?
Org
['alt.atheism', 'sci.space']
(1786,)


Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
info = tfidf.fit_transform(X)

In [106]:
feature_mapping = tfidf.get_feature_names()

Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

In [94]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(info, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [104]:
for a in gs.grid_scores_:
    print(a.mean_validation_score, a.parameters)

0.552631578947 {'C': 1.0000000000000001e-05}
0.552631578947 {'C': 0.0001}
0.552631578947 {'C': 0.001}
0.552631578947 {'C': 0.01}
0.950167973124 {'C': 0.10000000000000001}
0.993281075028 {'C': 1.0}
0.993281075028 {'C': 10.0}
0.993281075028 {'C': 100.0}
0.993281075028 {'C': 1000.0}
0.993281075028 {'C': 10000.0}
0.993281075028 {'C': 100000.0}


/home/ilya/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

In [131]:
best_clf = SVC(kernel='linear', random_state=241, C=1)
best_clf.fit(info, y)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [132]:
from scipy import sparse
dict_coef = sparse.dok_matrix(best_clf.coef_)

In [133]:
feature_with_imp = [(i[0][1], i[1]) for i in dict_coef.items()]
feature_with_imp = sorted(feature_with_imp, key=lambda x: abs(x[1]), reverse=True)

In [134]:
feature_with_imp[:3]

[(24019, 2.6631647884797105),
 (12871, -1.9203794002294938),
 (5088, -1.2546899512384038)]

Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

In [135]:
important_words = [feature_mapping[feature_with_imp[i][0]] for i in range(10)]
important_words = sorted(important_words)

In [136]:
with open('analisys_text.txt', 'w') as f:
    f.write(' '.join(important_words))
!cat analisys_text.txt

atheism atheists bible god keith moon religion sci sky space